In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.lines as mlines

from itertools import cycle
from matplotlib import pyplot as plt

from simulation import SimulationMixin


In [2]:
fontsize = 20

plots_dir = "plots"

In [3]:
colors = [("red", "red"), ("green", "green"), ("blue", "blue"), ("orange", "orange")]
color_cycle = cycle(colors)

In [4]:
# program = "graph_coloring"
program = "dijkstra_token_ring"
# program = "dijkstra"
# graph_name = "graph_powerlaw_cluster_graph_n7"
# graph_name = "implicit_graph_n8"
graph_name = "implicit_graph_n7"
sched = 0
no_simulations = 10000
me = False
fault_interval = 1

duong_mode = True

selected_nodes = [6]

In [5]:
def get_filename(graph_name, sched, simulation_type, args, no_simulations, me, fault_interval):
    return f"{graph_name}__{sched}__{simulation_type}_args_{args}__{no_simulations}__{me}__{fault_interval}"

In [6]:
# filenames = [
#     get_filename(
#         graph_name,
#         sched,
#         SimulationMixin.RANDOM_FAULT_SIMULATION_TYPE,
#         "",
#         no_simulations,
#         me,
#         fault_interval,
#     )
# ]
filenames = []
if duong_mode:
    filenames.extend(
        [
            get_filename(
                graph_name,
                sched,
                SimulationMixin.CONTROLLED_FAULT_AT_NODE_SIMULATION_TYPE_DUONG,
                arg,
                no_simulations,
                me,
                fault_interval,
            )
            for arg in selected_nodes
        ]
    )
else:
    filenames.extend(
        [
            get_filename(
                graph_name,
                sched,
                SimulationMixin.CONTROLLED_FAULT_AT_NODE_SIMULATION_TYPE,
                arg,
                no_simulations,
                me,
                fault_interval,
            )
            for arg in selected_nodes
        ]
    )


In [7]:
dfs = [pd.read_csv(os.path.join("results", program, f"{fn}.csv")) for fn in filenames]

In [8]:
max_steps = max(df['Steps'].max() for df in dfs)
max_steps

np.int64(561)

In [9]:
bins = np.linspace(3, max_steps, max_steps+1)
# bin_centers = (bins[:-1] + bins[1:]) / 2
bin_centers = bins

In [10]:
bins

array([  3.        ,   3.99465241,   4.98930481,   5.98395722,
         6.97860963,   7.97326203,   8.96791444,   9.96256684,
        10.95721925,  11.95187166,  12.94652406,  13.94117647,
        14.93582888,  15.93048128,  16.92513369,  17.9197861 ,
        18.9144385 ,  19.90909091,  20.90374332,  21.89839572,
        22.89304813,  23.88770053,  24.88235294,  25.87700535,
        26.87165775,  27.86631016,  28.86096257,  29.85561497,
        30.85026738,  31.84491979,  32.83957219,  33.8342246 ,
        34.82887701,  35.82352941,  36.81818182,  37.81283422,
        38.80748663,  39.80213904,  40.79679144,  41.79144385,
        42.78609626,  43.78074866,  44.77540107,  45.77005348,
        46.76470588,  47.75935829,  48.7540107 ,  49.7486631 ,
        50.74331551,  51.73796791,  52.73262032,  53.72727273,
        54.72192513,  55.71657754,  56.71122995,  57.70588235,
        58.70053476,  59.69518717,  60.68983957,  61.68449198,
        62.67914439,  63.67379679,  64.6684492 ,  65.66

In [11]:
hist_data = [np.histogram(df['Steps'], bins=bins) for df in dfs]
hist_data[0][0]

array([236, 198, 203, 203, 174, 171, 173, 201, 191, 166, 170, 181, 169,
       175, 181, 149, 145, 128, 115, 149, 131, 124, 136, 114, 119, 142,
       127, 116,  99, 104, 107,  99, 102,  97, 104,  96,  89,  96, 100,
        96,  62,  76,  74,  71,  66,  77,  74,  74,  71,  60,  69,  80,
        66,  64,  65,  62,  68,  68,  49,  62,  62,  49,  52,  60,  48,
        57,  53,  61,  48,  49,  37,  45,  44,  38,  45,  44,  39,  38,
        36,  37,  37,  46,  34,  36,  40,  29,  25,  32,  28,  33,  20,
        26,  29,  22,  17,  19,  22,  21,  32,  27,  21,  19,  22,  21,
        17,  19,  16,  17,  17,  19,  15,  21,  10,  20,  15,  21,  22,
        14,  18,  13,  15,  11,  16,   8,  12,  13,   7,  13,  18,   8,
        15,  11,  13,   7,  10,   8,  11,  12,  10,   4,   7,   6,   5,
         4,   8,   9,   4,   6,   6,   3,   8,  10,   9,   6,  10,  13,
         7,   9,   8,   3,   9,   6,   6,   6,   6,   4,   4,   2,   8,
         4,   0,   3,   4,   3,   7,   4,   4,   4,   3,   3,   

In [12]:
hist_data = [np.histogram(df['Steps'], bins=bins) for df in dfs]
hist_df = []
for hd in hist_data:
    hist_df.append(
        pd.DataFrame({
            'Steps': bin_centers[:-1],
            'Count': hd[0]
        })
    )

In [13]:
hist_df[0]

,Steps,Count
0,3.000000,236
1,3.994652,198
2,4.989305,203
3,5.983957,203
4,6.978610,174
...,...,...
556,556.026738,0
557,557.021390,0
558,558.016043,0
559,559.010695,0


In [14]:
df_merged = hist_df[0]
for i in range(1, len(hist_df)):
    df_merged = pd.merge(df_merged, hist_df[i], on=["Steps"], suffixes=(i-1, i))
df_merged

,Steps,Count
0,3.000000,236
1,3.994652,198
2,4.989305,203
3,5.983957,203
4,6.978610,174
...,...,...
556,556.026738,0
557,557.021390,0
558,558.016043,0
559,559.010695,0


In [15]:
df_merged.set_index('Steps', drop=True, inplace=True)
df_merged

,Count
Steps,
3.000000,236
3.994652,198
4.989305,203
5.983957,203
6.978610,174
...,...
556.026738,0
557.021390,0
558.016043,0


In [16]:
def get_title():
    return f"Simulation - {program} | {graph_name} | Sched: {sched} | N: {no_simulations:,} | FI: {fault_interval}"

In [17]:
def get_filename():
    return f"{program}__{graph_name}__{sched}__{no_simulations}__{fault_interval}__{''.join([str(i) for i in selected_nodes])}{'__duong' if duong_mode else ''}"

In [18]:
from scipy.optimize import curve_fit


def exp_func(x, a, b, c):
    return a * np.exp(-b * x) + c


# Fit the curve
popt, _ = curve_fit(exp_func, df_merged.index, df_merged["Count"], p0= [10, 0.5, 0])
y_fit = exp_func(df_merged.index, popt[0], popt[1], popt[2])
df_merged[f"exp_fit"] = y_fit

/home/agaru/anaconda3/envs/cvf/lib/python3.13/site-packages/pandas/core/indexes/base.py:952: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*new_inputs, **kwargs)


In [19]:
plt.figure(figsize=(16, 8))
ax = sns.lineplot(data=df_merged, linewidth=1)
ax.set_title(get_title(), fontdict={"fontsize": fontsize})

# ax.set_yscale("log")
for i, line in enumerate(ax.lines):
    if i >= 0:
        line_style = "solid"
        line.set_color(next(color_cycle)[0])
    else:
        line_style = "dashed"
        line.set_color("goldenrod")
    line.set_linestyle(line_style)

ax.tick_params(axis="x", labelsize=fontsize)
ax.tick_params(axis="y", labelsize=fontsize)
ax.xaxis.label.set_size(fontsize)
ax.yaxis.label.set_size(fontsize)

ax.set_xlabel("Steps")
ax.set_ylabel("Count")

# labels = ["Random Fault"]
labels = []
labels.extend(
    [
        f'Controlled {"(duong)" if duong_mode else ""} at node {n}'
        for n in selected_nodes
    ]
)
labels.append(f"$y = {popt[0]:.2f}e^{{{-popt[1]:.2f}x}} + {popt[1]:.2f}$")

custom_lines = [
    mlines.Line2D(
        [],
        [],
        color=line.get_color(),
        # marker=marker,
        label=cat,
        linestyle=line.get_linestyle(),
    )
    for line, cat in zip(ax.lines, labels)
]
plt.rc("font", size=fontsize)
plt.legend(handles=custom_lines, fontsize=fontsize * 0.9)
file_path = os.path.join(
    plots_dir,
    program,
    f"{get_filename()}.png",
)


plt.savefig(
    file_path,
    bbox_inches="tight",
)
plt.close()

print(f"Saved plot(s) for {file_path}")

Saved plot(s) for plots/dijkstra_token_ring/dijkstra_token_ring__implicit_graph_n7__0__10000__1__6__duong.png


In [20]:
df_merged

,Count,exp_fit
Steps,,
3.000000,236,212.775588
3.994652,198,208.020928
4.989305,203,203.372495
5.983957,203,198.827915
6.978610,174,194.384867
...,...,...
556.026738,0,-0.041430
557.021390,0,-0.041446
558.016043,0,-0.041462


In [21]:
# Save to file
cols = [
    f'Controlled {"(duong)" if duong_mode else ""} at node {n}' for n in selected_nodes
]
cols.append("exp_curve")
# df_merged.index = df_merged.index.astype(int)
df_merged.to_csv(os.path.join("results", program, f"w_exp_{get_filename()}.csv"))